In [4]:
import os
# Print number of files in each directory in /scratch/s194260/concepts_mmidb_tasks
for concept in os.listdir('/scratch/s194260/concepts_desync'):
    n_files = len(os.listdir(os.path.join('/scratch/s194260/concepts_desync', concept)))
    # Print those with motor in concept name
    if n_files >= 25:
        print(concept.ljust(75), n_files)

Alpha_Premotor Cortex-rh                                                    39
Alpha_Dorsal Stream Visual Cortex-rh                                        53
Alpha_Early Visual Cortex-rh                                                44
Alpha_Dorsal Stream Visual Cortex-lh                                        34
Alpha_Premotor Cortex-lh                                                    28
Alpha_Early Visual Cortex-lh                                                37
Alpha_MT+ Complex and Neighboring Visual Areas-rh                           131
Alpha_DorsoLateral Prefrontal Cortex-lh                                     254
Alpha_Orbital and Polar Frontal Cortex-rh                                   528
Alpha_Inferior Frontal Cortex-rh                                            546
Alpha_Superior Parietal Cortex-lh                                           95
Alpha_Auditory Association Cortex-lh                                        84
Alpha_Inferior Parietal Cortex-rh               

In [13]:
import os, random, shutil
from tqdm import tqdm
dir_path = "/scratch/s194260/concepts_mmidb_tasks"
# Copy all files in folders ending with lh into a the folder lh and all files ending with rh into the folder rh
for concept in tqdm(os.listdir(dir_path)):
    if concept.endswith('lh') and concept != 'lh':
        for file in os.listdir(os.path.join(dir_path, concept)):
            shutil.copy(os.path.join(dir_path, concept, file), os.path.join(dir_path, 'lh', file))
    elif concept.endswith('rh') and concept != 'rh':
        for file in os.listdir(os.path.join(dir_path, concept)):
            shutil.copy(os.path.join(dir_path, concept, file), os.path.join(dir_path, 'rh', file))

100%|██████████| 286/286 [00:01<00:00, 207.88it/s]


In [1]:

dir_path = "/nobackup/tsal-tmp/tuh_eeg"

count = 0

with tqdm(total=200) as pbar:
    # Choose 200 random .edf files from dir_path and its subfolders and copy them to /scratch/s194260/tuh_eeg
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".edf"):
                if random.random() < 0.01:
                    shutil.copy(os.path.join(root, file), '/scratch/s194260/tuh_eeg')
                    count += 1
                    pbar.update(1)
                    if count >= 200:
                        break
                    
        if count >= 200:
            break

100%|██████████| 200/200 [01:10<00:00,  2.84it/s]


In [33]:
def pick_and_rename_channels(raw):
    if 'P7' in raw.ch_names:
        raw.rename_channels({'P7': 'T5'})
    if 'P8' in raw.ch_names:
        raw.rename_channels({'P8': 'T6'})
    if 'T3' in raw.ch_names:
        raw.rename_channels({'T3': 'T7'})
    if 'T4' in raw.ch_names:
        raw.rename_channels({'T4': 'T8'})

    EEG_20_div = [
                'FP1', 'FP2',
        'F7', 'F3', 'FZ', 'F4', 'F8',
        'T7', 'C3', 'CZ', 'C4', 'T8',
        'T5', 'P3', 'PZ', 'P4', 'T6',
                 'O1', 'O2'
    ]
    
    raw.pick_channels(ch_names=EEG_20_div)
    raw.reorder_channels(EEG_20_div)

    return raw

In [34]:
def rename(name):
    name = name.rstrip('.').upper()
    name = name.replace("EEG ", "")
    name = name.replace("-REF", "")
    return name

In [ ]:
import os, random, shutil
from tqdm import tqdm
import mne
import os
import numpy as np
from tqdm import tqdm

dir_path = "/nobackup/tsal-tmp/tuh_eeg"

count = 0

with tqdm() as pbar:
    # Choose 200 random .edf files from dir_path and its subfolders and copy them to /scratch/s194260/tuh_eeg
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".edf"):
                if random.random() < 0.01:
                    try:
                        raw = mne.io.read_raw_edf(os.path.join(root, file), verbose=False, preload=True)
                        mne.datasets.eegbci.standardize(raw)  # Set channel names
                        new_names = dict((ch_name, rename(ch_name)) for ch_name in raw.ch_names)
                        raw.rename_channels(new_names)   
                        raw = raw.set_eeg_reference(ref_channels='average', projection=True, verbose = False)
                        raw = raw.apply_proj(verbose=False)

                        montage = mne.channels.make_standard_montage('standard_1020')
                        raw = raw.set_montage(montage, on_missing='ignore'); # Set montage
                        raw = pick_and_rename_channels(raw)
                        
                        # Save the raw data in /scratch/s194260/tuh_eeg_preprocessed as edf file
                        raw.export(os.path.join('/scratch/s194260/tuh_eeg_preprocessed', file), verbose=False, overwrite=True)
                        pbar.update(1)           
                        
                        
                    except Exception as e:
                        continue

In [ ]:
with tqdm(total=200) as pbar:
    for root, dirs, files in os.walk(dir_path):

In [62]:
# Go through each EDF file in /scratch/s194260/tuh_eeg and print its length in seconds
import mne
import os
import numpy as np
from tqdm import tqdm

dir_path = "/scratch/s194260/tuh_eeg"

with tqdm(total=200) as pbar:
    for file in os.listdir(dir_path):
        try:
            raw = mne.io.read_raw_edf(os.path.join(dir_path, file), verbose=False, preload=True)
            mne.datasets.eegbci.standardize(raw)  # Set channel names
            new_names = dict((ch_name, rename(ch_name)) for ch_name in raw.ch_names)
            raw.rename_channels(new_names)   
            raw = raw.set_eeg_reference(ref_channels='average', projection=True, verbose = False)
            raw = raw.apply_proj(verbose=False)

            montage = mne.channels.make_standard_montage('standard_1020')
            raw = raw.set_montage(montage, on_missing='ignore'); # Set montage
            raw = pick_and_rename_channels(raw)
            
            # Save the raw data in /scratch/s194260/tuh_eeg_preprocessed as edf file
            raw.export(os.path.join('/scratch/s194260/tuh_eeg_preprocessed', file), verbose=False, overwrite=True)
            pbar.update(1)           
            
            
        except Exception as e:
            print(file, e)
            print()

  1%|          | 2/200 [00:26<37:43, 11.43s/it]  

aaaaagxq_s001_t001.edf No channels match the selection.



  2%|▎         | 5/200 [00:34<14:25,  4.44s/it]

aaaaakyi_s002_t076.edf No channels match the selection.



  8%|▊         | 17/200 [01:19<07:43,  2.53s/it]

...

 75%|███████▌  | 150/200 [09:59<02:01,  2.43s/it]

aaaaaiql_s006_t001.edf No channels match the selection.



 78%|███████▊  | 156/200 [10:26<02:26,  3.32s/it]

aaaaadgv_s001_t001.edf No channels match the selection.

aaaaagyl_s001_t002.edf No channels match the selection.



 80%|███████▉  | 159/200 [10:29<02:42,  3.96s/it]
